In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
from tensorflow.keras.layers import Input, Dense, Concatenate
import ast

In [2]:
model = tf.keras.models.load_model("v3.keras")
ep=150
bs=1024
model.summary()

Model: "functional_2"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_7       │ (None, 384)       │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_8       │ (None, 384)       │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_15 (Dense)    │ (None, 500)       │    192,500 │ input_layer_7[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_17 (Dense)    │ (None, 500)       │    192,500 │ input_layer_8[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_6       │ (None, 5)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_16 (Dense)    │ (None, 250)       │    125,250 │ dense_15[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_18 (Dense)    │ (None, 250)       │    125,250 │ dense_17[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_2       │ (None, 505)       │          0 │ input_layer_6[0]… │
│ (Concatenate)       │                   │            │ dense_16[0][0],   │
│                     │                   │            │ dense_18[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_19 (Dense)    │ (None, 250)       │    126,500 │ concatenate_2[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_20 (Dense)    │ (None, 244)       │     61,244 │ dense_19[0][0]    │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 2,469,734 (9.42 MB)

 Trainable params: 823,244 (3.14 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 1,646,490 (6.28 MB)

In [ ]:
df_train=pd.read_csv("sep_selected_laws_final_train.csv")
df_val=pd.read_csv("sep_selected_laws_final_val.csv")

#language_train=df_train.values[:,:3]
law_area_train=df_train.values[:,3:8]
facts_train=df_train.values[:,8:392]
considerations_train=df_train.values[:,392:776]
laws_train=df_train.values[:,776:]


#language_val=df_val.values[:,:3]
law_area_val=df_val.values[:,3:8]
facts_val=df_val.values[:,8:392]
considerations_val=df_val.values[:,392:776]
laws_val=df_val.values[:,776:]

In [17]:
def compare_output_of_row(i):
    t= ([np.array([law_area_val[i].tolist()]), np.array([facts_val[i].tolist()]), np.array([considerations_val[i].tolist()])]  ,  np.array([laws_val[i].tolist()]))
    print("prediction:")
    print((model(t[0])>0.5).numpy()[0].astype(int))
    print("actual:")
    print(t[1][0].astype(int))
compare_output_of_row(5)

prediction:
[1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
actual:
[1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0

In [5]:
def calculate_recprec(actual, prediction):
    recall=[]
    precision=[]
    total_laws=actual[0].shape[0]
    for law in range(total_laws):
        recaller=tf.keras.metrics.Recall()
        recaller.update_state(actual[:,law].tolist(),prediction[:,law].tolist())
        recall.append(float(recaller.result().numpy()))
        
        precisioner=tf.keras.metrics.Precision()
        precisioner.update_state(actual[:,law].tolist(),prediction[:,law].tolist())
        precision.append(float(precisioner.result().numpy()))
    
    '''
    print("Individual recall:")
    print(recall)
    print("Individual precision:")
    print(precision)
    '''
    
    print("Average recall:", sum(recall)/total_laws)
    print("Average precision:", sum(precision)/total_laws)

In [6]:
train_prediction=(model.predict([law_area_train, facts_train, considerations_train],batch_size=1024) > 0.5).astype(int)
val_prediction=(model.predict([law_area_val, facts_val, considerations_val],batch_size=1024) > 0.5).astype(int)

print("Training set:")
calculate_recprec(laws_train, train_prediction)
print("Validdation set:")
calculate_recprec(laws_val, val_prediction)
print("Epochs:",ep)
print("Batch size:",bs)

76/76 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
Training set:
Average recall: 0.9972354146789332
Average precision: 0.9991444373717073
Validdation set:
Average recall: 0.23318577614292257
Average precision: 0.3090214709537562
Epochs: 150
Batch size: 1024
